<a href="https://colab.research.google.com/github/badjuju0/miigaikPractice/blob/main/Zadanie3/%D0%9D%D0%B5%D0%B4%D0%B5%D0%BB%D1%8F_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Неделя 3. Перепроецирование с использованием pyproj
## Системы координат. Немного теории

*К сожалению, Земля не плоская. А монитор плоский*

Почитать краткую теорию про системы координат вообще можно вот здесь https://disk.yandex.ru/i/vY__bZq5vumQoQ . Эта информация не понадобится для выполнения работы напрямую, но облегчит понимание того, что мы вообще делаем и зачем.

Большинство карт по умолчанию работают с wgs84 (EPSG:4326 https://epsg.io/4326), когда идет речь о координатах в градусах. Также часто используется EPSG:3857  https://epsg.io/3857, если необходимо описание координат в метрах.

Кадастр в РФ работает с координатами характерных точек в местных системах координат (МСК). Местная система координат - это плоская система, взятая как проекция Меркатора к эллипсоиду Красовского. Для того, чтобы минимизировать погрешности определения точек, РФ поделена на зоны картографической проекции, объекты в каждой из которой ставятся на учет в соответствующей этой зоне системе координат. Каждый регион разделен на 3-градусные зоны. Разделение проходит не по прямым, параллельным меридианам, а вдоль кадастрового деления. Что дает нам факт: каждый кадастровый квартал должен содержать внутри объекты в одной системе координат. Внутри квартала не может существовать двух рекомендуемых местных систем координат.

### Формат строки proj

Существует множество форматов, которые используются для описания референсных систем координат. Наиболее распространенными форматами являются: proj.4, EPSG, формат Well-known Text (WKT).

Для работы с pyproj нам необходимо использовать формат proj.4

Формат PROJ.4 strings – используют многие программные продукты для обработки пространственных данных, (например Raster, Rgdal, Qgis). Используя PROJ.4 синтаксис, указываетcя полный
набор параметров, включая эллипс, датум, единицы проекции и определение проекции, которые определяют конкретную референсную систему координат.
Каждый элемент обозначается знаком «+». После каждого «+» задается определяемый элемент.
Например +proj= и +datum=.
Строка proj4 собирается из отдельных компонентов (разделенные знаком «+»). Например:
- proj = utm: проекция UTM
- zone = 11: зона 11.
- datum = WGS84: начало отсчета относится к опорной точке 0,0 для системы координат,
используемой в проекции
- units = m: координаты указаны в МЕТРАХ.
- ellps = WGS84: эллипсоид для данных - WGS84

В результате получаем строку:
`+proj=utm +zone=11 +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0 `

Для описания других систем координат этих параметров недостаточно, и добавляются следующие:
- lat_0 = 0  начало координат осевого меридиана.
- lon_0 = 44,55 долгота осевого меридиана.
- k = 1 масштаб на осевом меридиане (для проекции Гаусса- Крюгера равен 1)
- x_0 = 1250000 ложное отклонение от меридиана к востоку в метрах (False easting)
- y_0 =-5614743,504 ложное отклонение от широты к северу в метрах (False northing)
  
То есть, для пересчета  мы можем использовать строки вида `+proj=tmerc +lat_0=0 +lon_0=44.55 +k=1 +x_0=1250000 +y_0=-5614743.504 +ellps=krass +units=m +no_defs`

Почитать подробнее про параметры (на английском) можно вот здесь - https://proj.org/en/9.3/usage/projections.html

Для системы координат EPSG:4326 строка параметров выглядит `+proj=longlat +datum=WGS84 +no_defs +type=crs`, а для EPSG:3857 выглядит `+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs +type=crs`


## Знакомимся с библиотекой proj
### Установим библиотеки
Если вы выполняли все задания в одной и той же среде, то остальные нужные библиотеки у вас уже установлены.
Если какой-то из фрагментов кода не запускается и ругается на отсутствие библиотеки, скопируйте блок с её установкой из предыдущих работ и запустите.

In [ ]:
!pip install pyproj

Defaulting to user installation because normal site-packages is not writeable


### Пересчет. Простой случай
Если вы используете одно и то же преобразование, использование `pyproj.transformer.Transformer` может помочь оптимизировать ваши преобразования.
Системы координат можно описать используя коды EPSG

In [ ]:
import numpy
from pyproj import Transformer, transform

transformer = Transformer.from_crs(2263, 4326)
x_coords = numpy.random.randint(80000, 120000)
y_coords = numpy.random.randint(200000, 250000)
transformer.transform(x_coords, y_coords)

(40.766428579394535, -77.24473209793454)

### Запуск преобразований. Описания параметров преобразования
Если хочется исследовать преобразования подробнее, можно почитать вот тут: https://pyproj4.github.io/pyproj/stable/advanced_examples.html

https://pyproj4.github.io/pyproj/stable/api/transformer.html

`pyproj.transformer.TransformerGroup` предоставляет как доступные, так и отсутствующие преобразования. Полезно, если вы хотите проверить, существует ли наилучшее возможное преобразование. А если нет, то как получить недостающую сетку.
Кроме кодов, вы также можете описать систему координат с помощью строки proj.

In [ ]:
from pyproj.transformer import TransformerGroup
tg = TransformerGroup("EPSG:4326", "+proj=aea +lat_0=50 +lon_0=-154 +lat_1=55 +lat_2=65 +x_0=0 +y_0=0 +datum=NAD27 +no_defs +type=crs +units=m", always_xy=True)
tg

<TransformerGroup: best_available=False>
- transformers: 31
- unavailable_operations: 48

In [ ]:
from pyproj.transformer import TransformerGroup
tg = TransformerGroup("EPSG:4326", "+proj=aea +lat_0=50 +lon_0=-154 +lat_1=55 +lat_2=65 +x_0=0 +y_0=0 +datum=NAD27 +no_defs +type=crs +units=m", always_xy=True)
tg.transformers[0].description

'axis order change (2D) + Inverse of NAD27 to WGS 84 (3) + axis order change (2D) + unknown'

In [ ]:
from pyproj.transformer import TransformerGroup
tg = TransformerGroup("EPSG:4326", "+proj=aea +lat_0=50 +lon_0=-154 +lat_1=55 +lat_2=65 +x_0=0 +y_0=0 +datum=NAD27 +no_defs +type=crs +units=m", always_xy=True)
tg.unavailable_operations[0].name

'Inverse of NAD27 to WGS 84 (33) + axis order change (2D) + unknown'

In [ ]:
from pyproj.transformer import TransformerGroup
tg = TransformerGroup("EPSG:4326", "+proj=aea +lat_0=50 +lon_0=-154 +lat_1=55 +lat_2=65 +x_0=0 +y_0=0 +datum=NAD27 +no_defs +type=crs +units=m", always_xy=True)
tg.unavailable_operations[0].grids[0].url

'https://cdn.proj.org/ca_nrc_ntv2_0.tif'

### Пересчет объекта целиком

Для запуска пересчета можно подать точки целым массивом или обратиться ко всем точкам геометрического объекта.


In [ ]:
from pyproj import Transformer
from shapely.geometry import LineString, Point, Polygon
transformer = Transformer.from_crs(4326, 2100)
points = [(22.95, 40.63), (22.81, 40.53), (23.51, 40.86)]
print("Пересчет из EPSG:4326 (градусная wgs84) в EPSG:2100: \n")
for pt in transformer.itransform(points):
  print(pt)
print("\n Пересчет из EPSG:4326 в некую МСК: \n")
transproj_eq = Transformer.from_crs(
    'EPSG:4326',
    '+proj=tmerc +lat_0=0 +lon_0=44.55 +k=1 +x_0=1250000 +y_0=-5614743.504 +ellps=krass +units=m +no_defs',
    always_xy=True,
)
for pt in transproj_eq.itransform(points):
    print(pt)
print("\n Пересчет из EPSG:4326 в некую МСК, с обращением к полигону: \n")
pol =     Polygon(
        LineString([
            Point(37.660516, 55.763809), Point(37.659749, 55.764129), Point(37.659266, 55.763821), Point(37.65962, 55.76329), Point(37.660156, 55.763392), Point(37.660516, 55.763809)
        ])
    )
for point in transproj_eq.itransform(pol.exterior.coords):
  print(point)

Пересчет из EPSG:4326 (градусная wgs84) в EPSG:2100: 

(2221638.800587292, 2637034.3720682357)
(2212924.124633719, 2619851.898196797)
(2238294.779022731, 2703763.7362801754)

 Пересчет из EPSG:4326 в некую МСК: 

(-583310.9199796189, -884216.8687568726)
(-598165.1278206981, -892327.415623053)
(-529065.0846791547, -870617.2738845842)

 Пересчет из EPSG:4326 в некую МСК, с обращением к полигону: 

(817904.9069238179, 589149.9011352416)
(817860.4380617549, 589190.2314159079)
(817826.7826442412, 589159.0516230008)
(817843.0544412159, 589097.8743061014)
(817877.7431213218, 589105.8481806554)
(817904.9069238179, 589149.9011352416)


### Методы для вычисления периметра и площади
А еще эта библиотека позволяет встроенными методами считать площадь и периметр объектов, опираясь на выбранный эллипсоид.

In [ ]:
from pyproj import Geod
from shapely.geometry import LineString, Point, Polygon
from IPython.display import HTML, display
geod = Geod(ellps="WGS84")
pol =     Polygon(
        LineString([
            Point(37.660516, 55.763809), Point(37.659749, 55.764129), Point(37.659266, 55.763821), Point(37.65962, 55.76329), Point(37.660156, 55.763392), Point(37.660516, 55.763809)
        ]),
        holes=[LineString([Point(37.659878, 55.763917), Point(37.660221, 55.763803), Point(37.659802, 55.763507), Point(37.659878, 55.763917)])],
    )
poly_area, poly_perimeter = geod.geometry_area_perimeter(pol)
s = "Площадь полигона с дыркой: " + str(round(poly_area, 2)) + " кв. метров, периметр полигона: " + str(round(poly_perimeter, 2)) + " метров. "
print(s)
html_pol =pol._repr_svg_()
print(html_pol)
display(HTML(html_pol))

Площадь полигона с дыркой: 3801.29 кв. метров, периметр полигона: 255.97 метров. 
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="100.0" height="100.0" viewBox="37.659216 55.763239999999996 0.0013500000000021828 0.0009390000000024656" preserveAspectRatio="xMinYMin meet"><g transform="matrix(1,0,0,-1,0,111.527419)"><path fill-rule="evenodd" fill="#66cc99" stroke="#555555" stroke-width="2.7000000000043655e-05" opacity="0.6" d="M 37.660516,55.763809 L 37.659749,55.764129 L 37.659266,55.763821 L 37.65962,55.76329 L 37.660156,55.763392 L 37.660516,55.763809 z M 37.659878,55.763917 L 37.660221,55.763803 L 37.659802,55.763507 L 37.659878,55.763917 z" /></g></svg>


## Приступим!
В следующем фрагменте кода реализована генерация псевдослучайного полигона вокруг стартовой точки. И заложена основа для формирования отчета.
Последняя строка выводит картинку получившегося полигона ниже кода, и нужна только для отладки. На файл отчета она не влияет.

In [1]:
from pyproj import Geod
from shapely.geometry import LineString, Point, Polygon
from pyproj import Transformer
from IPython.display import HTML, display
import random
import math
# задаем стартовую точку
startPoint = Point(55.864530, 37.660629)
# генерация полигона - начало
lst = []
j = 1
count_p = 10 + round(7*random.random())
center_x = startPoint.x
center_y = startPoint.y
while j < count_p:
    radius = 0.0021*(0.1+random.random())
    angle = j*2*math.pi/count_p
    lat = (center_x+radius*math.cos(angle))
    lon = (center_y+radius*math.sin(angle))
    lst.append([lat, lon])
    j = j+1
polygon = Polygon(lst)
# генерация полигона - конец
result_html='''<html>
    <body>
      <h1>Отчет за 3 неделю ознакомительной практики по разработке программного обеспечения геосервиса</h1>
      <h2>Выполнил: студент группы 2024-ИСиТ-1см: Горбачев  П.А.</h2>'''
      # здесь вставить формирование svg
svg = polygon._repr_svg_()
# записать полученное svg в конце result_html
result_html += "<h3>Полигон в формате .svg</h3>" + svg
  # здесь вставить подсчет площади и подсчет периметра
geod = Geod(ellps="WGS84")
area, perimeter = geod.geometry_area_perimeter(polygon)
# записать площадь и периметр в конце result_html
result_html += f"<h3>Площадь полигона:</h3> {abs(area):.2f} м²"
result_html += f"<h3>Периметр полигона:</h3> {perimeter:.2f} м"
  # здесь вставить пересчет всех точек
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
transformed_coords = [transformer.transform(x, y) for x, y in polygon.exterior.coords]
# записать сведения о системе координат и координаты пересчитанных точек в конце result_html
result_html += "<h3>Пересчет координат</h3>"
result_html += "<p>Исходная система: EPSG:4326<br>Целевая система: EPSG:3857</p>"
result_html += "<pre>" + str(transformed_coords) + "</pre>"

result_html = result_html + ('''
    </body>
</html>''')
file_html = open("result_3.html", "w")
file_html.write(result_html)
file_html.close()
display(HTML(polygon._repr_svg_()))


### Задание
1. Добавить титул с названием отчета, ФИО и группой
2. Изменить стартовую точку на ваше текущее местоположение. Добавить в отчет картинку полигона (svg)
3. Добавить в формируемый отчет подсчет площади и подсчет периметра генерируемого полигона
4. Добавить в отчет указание целевой системы координат и массив пересчитанных точек. По умолчанию пересчитываем из EPSG:4326 в EPSG:3857
5. Для выполнения пунктов 2-4 вносите изменения в предыдущий блок кода. Результат будет сохранен в файл `result_3.html`, его и приложить в СДО.